<a href="https://colab.research.google.com/github/k-vinamr/new_rera/blob/main/new_5th_floor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import joblib

In [81]:
def load_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)
    return data

In [82]:
def preprocess_data(data):
    # Convert date columns to datetime objects
    data['actual_commencement_date'] = pd.to_datetime(data['actual_commencement_date'], errors='coerce')
    data['estimated_finish_date'] = pd.to_datetime(data['estimated_finish_date'], errors='coerce')

    # Calculate derived columns
    current_date = datetime.now()
    data['duration_since_commencement'] = (current_date - data['actual_commencement_date']).dt.days
    data['remaining_duration'] = (data['estimated_finish_date'] - current_date).dt.days
    data['progress_ratio'] = data['duration_since_commencement'] / data['total_days']
    data['current_stage'] = data.apply(determine_status, axis=1)

    # Label encoding for 'current_stage' and 'project_state' columns
    label_encoder = LabelEncoder()
    data['current_stage'] = data['current_stage'].astype(str)
    data['current_stage_encoded'] = label_encoder.fit_transform(data['current_stage'])
    data['project_state'] = data['project_state'].astype(str)
    data['project_state_encoded'] = label_encoder.fit_transform(data['project_state'])

    return data

In [83]:
# Define a function to determine the project status
def determine_status(row):
    current_date = datetime.now()
    if current_date > row['estimated_finish_date']:
        return 'completed'
    elif current_date >= row['actual_commencement_date']:
        return 'running'
    else:
        return 'upcoming'

# Apply the function to create the 'Status' column
    data['current_stage'] = data.apply(determine_status, axis=1)
    return data

In [84]:
def feature_selection(data):
    # Select relevant features
    numerical_columns = data.select_dtypes(include=['int64', 'float64'])
    stage_columns = [
        'Cleaning & survey',
        'Excavation,leveling & P.C.C for Basement B1',
        'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
        'Slab of B (bottom)',
        'Ground Floor slab casting',
        '1st floor Columns casting',
        '1st Floor slab casting',
        '2nd floor Columns casting',
        '2nd Floor slab casting',
        'Brick work at Basement to Ground Floor',
        '3rd floor Columns casting',
        '3rd Floor slab casting',
        'Brick work at 1st Floor',
        'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
        '4th floor Columns casting',
        '4th Floor slab casting',
        'Brick work at 2nd Floor',
        'Electrical concealed, PVC Fitting, plastering at 1st floor',
        '5th floor Columns casting',
        '5th Floor slab casting',
        'Brick work of 3rd to 5th Floor',
        'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
        'Plastering on outer sides',
        'Tiles work',
        'Painting and Finishing',
        'Plumbing & Sanitary,Electrification Works',
        'Doors & Windows Fixing Furniture work'
    ]

    X = data[numerical_columns.columns.difference(stage_columns)].drop(columns=['floor', 'total_area_of_project'])
    y_time_taken_at_each_stage = data[stage_columns]

    return X, y_time_taken_at_each_stage

In [85]:
def train_stage_models(X, y_stages, stage_columns):
    stage_models = {}
    performance_metrics = {}

    for stage in stage_columns:
        X_train, X_test, y_train, y_test = train_test_split(X, y_stages[stage], test_size=0.2, random_state=42)
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        stage_models[stage] = model
        performance_metrics[stage] = {'MSE': mse, 'R2': r2}

    return stage_models, performance_metrics

In [102]:
def determine_relevant_stage_and_columns(row, stage_columns):
    current_date = datetime.now()
    days_elapsed = row['days_elapsed']
    total_day = 0
    relevant_stage = None
    relevant_columns = []

    for stage in stage_columns:
        stage_duration = row[stage]  # Days required for the current stage
        total_day += stage_duration
        if total_day >= days_elapsed:
            relevant_stage = stage
            relevant_columns = [stage for stage in stage_columns[stage_columns.index(stage):]]
            break

    return relevant_stage, relevant_columns, total_day

In [103]:
def make_predictions(row, stage_columns):
    if row['current_stage'] == 'completed':
        return None  # No predictions for completed projects
    elif row['current_stage'] == 'running':
        if row['relevant_stage'] is not None:
            if row['relevant_stage'] in row['relevant_columns']:
                return row[row['relevant_stage']] - (row['days_elapsed'] - row['total_days'])
            else:
                return None  # Skip stages before the relevant stage
        else:
            return None  # Skip predictions for running projects with no relevant stage
    elif row['current_stage'] == 'upcoming':
        return row['total_days']  # Include all stages for upcoming projects

In [104]:
def predict_stage_durations(data, stage_models, stage_columns):
    """
    Predict the duration for each stage using the respective models.
    """
    for stage in stage_columns:
        model = stage_models.get(stage)
        if model:
            data[stage] = model.predict(data)
        else:
            data[stage] = np.nan
    return data

In [105]:
import numpy as np

def adjust_stage_durations(data, stage_columns, max_duration_col='estimated_duration_days', round_decimals=2):
    for index, row in data.iterrows():
        total_duration = row[max_duration_col]
        stage_durations = row[stage_columns]

        # Handle NaN or non-numeric values
        stage_durations = stage_durations.apply(lambda x: x if np.isfinite(x) and np.isscalar(x) else 0)

        # Calculate the cumulative duration
        cumulative_duration = stage_durations.sum()

        if cumulative_duration > total_duration:
            # Calculate the proportion of each stage to the total
            proportions = stage_durations / cumulative_duration
            excess_duration = cumulative_duration - total_duration

            # Adjust each stage's duration
            adjusted_durations = stage_durations - excess_duration * proportions
            adjusted_durations = adjusted_durations.clip(lower=0)  # Ensuring no negative durations
            adjusted_durations = adjusted_durations.round(round_decimals)  # Rounding to handle minor errors

            data.loc[index, stage_columns] = adjusted_durations

        # Validation check
        if not np.isclose(data.loc[index, stage_columns].sum(), total_duration, atol=0.01):
            print(f"Warning: Total adjusted durations for Project {index + 1} do not match total_duration.")

        # Adjust the last stage duration if final stage end date exceeds estimated finish date
        final_stage_end_date = row['actual_commencement_date'] + pd.Timedelta(days=data.loc[index, stage_columns].sum())
        if final_stage_end_date > row['estimated_finish_date']:
            final_stage_duration = (row['estimated_finish_date'] - row['actual_commencement_date']).days
            final_stage_adjusted_duration = final_stage_duration - data.loc[index, stage_columns[:-1]].sum()
            data.at[index, stage_columns[-1]] = max(0, final_stage_adjusted_duration)

        print(f"Project {index + 1}: Adjusted Durations - {data.loc[index, stage_columns].tolist()}")

    return data

In [106]:
def main(file_path):
    data = load_data(file_path)
    data['total_days'] = data['estimated_duration_days']
    data = preprocess_data(data)

    # Determine relevant stage and columns
    data['current_stage'] = data.apply(determine_status, axis=1)

    X, y_time_taken_at_each_stage = feature_selection(data)

    stage_columns = [
        'Cleaning & survey',
        'Excavation,leveling & P.C.C for Basement B1',
        'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
        'Slab of B (bottom)',
        'Ground Floor slab casting',
        '1st floor Columns casting',
        '1st Floor slab casting',
        '2nd floor Columns casting',
        '2nd Floor slab casting',
        'Brick work at Basement to Ground Floor',
        '3rd floor Columns casting',
        '3rd Floor slab casting',
        'Brick work at 1st Floor',
        'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
        '4th floor Columns casting',
        '4th Floor slab casting',
        'Brick work at 2nd Floor',
        'Electrical concealed, PVC Fitting, plastering at 1st floor',
        '5th floor Columns casting',
        '5th Floor slab casting',
        'Brick work of 3rd to 5th Floor',
        'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
        'Plastering on outer sides',
        'Tiles work',
        'Painting and Finishing',
        'Plumbing & Sanitary,Electrification Works',
        'Doors & Windows Fixing Furniture work'
    ]


    # Split the data into training and testing sets for time taken at each stage
    X_train_time, X_test_time, y_train_time, y_test_time = train_test_split(X, y_time_taken_at_each_stage, test_size=0.2, random_state=42)

    stage_models = {}
    for stage in stage_columns:
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train_time, y_train_time[stage])
        stage_models[stage] = model

        # Predict time taken at each stage on the test set
        y_pred = model.predict(X_test_time)
        mse = mean_squared_error(y_test_time[stage], y_pred)
        r2 = r2_score(y_test_time[stage], y_pred)
        print(f'Mean Squared Error ({stage}): {mse}')
        print(f'R-squared ({stage}): {r2}')

    # Predict stage durations for the entire dataset
    for stage in stage_columns:
        data[stage] = stage_models[stage].predict(data[X.columns])

    # Adjust stage durations
    data = adjust_stage_durations(data, stage_columns)

    # Additional logic for determining relevant stages and making predictions
    current_date = datetime.now()
    data['days_elapsed'] = (current_date - data['actual_commencement_date']).dt.days
    data[['relevant_stage', 'relevant_columns', 'total_day']] = data.apply(
        lambda row: determine_relevant_stage_and_columns(row, stage_columns), axis=1, result_type='expand')

    # Filter projects based on current_stage and make predictions
    data['predicted_time_taken'] = data.apply(lambda row: make_predictions(row, stage_columns), axis=1)
    data = data.dropna()

    # Save models and adjusted data
    for stage, model in stage_models.items():
        joblib.dump(model, f'{stage}_model.joblib')
    data.to_csv('adjusted_project_stages.csv', index=False)

    return data

if __name__ == "__main__":
    file_path = 'updated_5th_floor_with_stages.csv'
    main(file_path)

Mean Squared Error (Cleaning & survey): 0.002096950213424238
R-squared (Cleaning & survey): 0.9997317616724766
Mean Squared Error (Excavation,leveling & P.C.C for Basement B1): 0.03304352560964871
R-squared (Excavation,leveling & P.C.C for Basement B1): 0.9997358205029062
Mean Squared Error (Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring): 0.039162357325030865
R-squared (Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring): 0.9998282036157097
Mean Squared Error (Slab of B (bottom)): 0.014176508992176356
R-squared (Slab of B (bottom)): 0.9997985072141734
Mean Squared Error (Ground Floor slab casting): 0.06536930238456959
R-squared (Ground Floor slab casting): 0.9997333568020078
Mean Squared Error (1st floor Columns casting): 0.008692640156285717
R-squared (1st floor Columns casting): 0.9997702587110512
Mean Squared Error (1st Floor slab casting): 0.03304352560964871
R-squared (1st Floor slab casting): 0.9997358205029062
Mean Squared Error (2n

In [111]:
# Load the new data (replace 'new_data.csv' with the actual file path)
new_data = pd.read_csv('5th_floor.csv')

stage_columns = [
    'Cleaning & survey',
    'Excavation,leveling & P.C.C for Basement B1',
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
    'Slab of B (bottom)',
    'Ground Floor slab casting',
    '1st floor Columns casting',
    '1st Floor slab casting',
    '2nd floor Columns casting',
    '2nd Floor slab casting',
    'Brick work at Basement to Ground Floor',
    '3rd floor Columns casting',
    '3rd Floor slab casting',
    'Brick work at 1st Floor',
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
    '4th floor Columns casting',
    '4th Floor slab casting',
    'Brick work at 2nd Floor',
    'Electrical concealed, PVC Fitting, plastering at 1st floor',
    '5th floor Columns casting',
    '5th Floor slab casting',
    'Brick work of 3rd to 5th Floor',
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
    'Plastering on outer sides',
    'Tiles work',
    'Painting and Finishing',
    'Plumbing & Sanitary,Electrification Works',
    'Doors & Windows Fixing Furniture work'
]


# Load each stage model
stage_models = {}
for stage in stage_columns:
    stage_model = joblib.load(f'{stage}_model.joblib')
    stage_models[stage] = stage_model

# Converting date columns to datetime objeacts
new_data['actual_commencement_date'] = pd.to_datetime(new_data['actual_commencement_date'], errors='coerce')
new_data['estimated_finish_date'] = pd.to_datetime(new_data['estimated_finish_date'], errors='coerce')

current_date = datetime.now()

new_data['estimated_duration_days'] = (new_data['estimated_finish_date'] - new_data['actual_commencement_date']).dt.days
new_data['total_days'] = new_data['estimated_duration_days']
new_data['duration_since_commencement'] = (current_date - new_data['actual_commencement_date']).dt.days
new_data['remaining_duration'] = (new_data['estimated_finish_date'] - current_date).dt.days
new_data['progress_ratio'] = new_data['duration_since_commencement'] / new_data['total_days']

# Define a function to determine the project status
def determine_status(row):
    if current_date > row['estimated_finish_date']:
        return 'completed'
    elif current_date >= row['actual_commencement_date']:
        return 'running'
    else:
        return 'upcoming'

# Apply the function to create the 'Status' column
new_data['current_stage'] = new_data.apply(determine_status, axis=1)

# Performing label encoding on the 'current_stage' column
label_encoder = LabelEncoder()
new_data['current_stage'] =new_data['current_stage'].astype(str) # Converting to string to handle any NaN values left
new_data['current_stage_encoded'] = label_encoder.fit_transform(new_data['current_stage'])

# Performing label encoding on the 'current_stage' column
label_encoder = LabelEncoder()
new_data['project_state'] =new_data['project_state'].astype(str) # Converting to string to handle any NaN values left
new_data['project_state_encoded'] = label_encoder.fit_transform(new_data['project_state'])

new_data
# Create an empty DataFrame to store 'new_data' with added columns
new_data = new_data.copy()  # Assuming 'new_data' is your original DataFrame

# Add stage-related columns and 'total_days' column with NaN values
for stage in stage_columns:
    new_data[stage] = np.nan

# Now, 'new_data_with_stages' contains the additional columns with NaN values

numerical_columns = new_data.select_dtypes(include=['int64', 'float64'])
stage_columns = [
    'Cleaning & survey',
    'Excavation,leveling & P.C.C for Basement B1',
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
    'Slab of B (bottom)',
    'Ground Floor slab casting',
    '1st floor Columns casting',
    '1st Floor slab casting',
    '2nd floor Columns casting',
    '2nd Floor slab casting',
    'Brick work at Basement to Ground Floor',
    '3rd floor Columns casting',
    '3rd Floor slab casting',
    'Brick work at 1st Floor',
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
    '4th floor Columns casting',
    '4th Floor slab casting',
    'Brick work at 2nd Floor',
    'Electrical concealed, PVC Fitting, plastering at 1st floor',
    '5th floor Columns casting',
    '5th Floor slab casting',
    'Brick work of 3rd to 5th Floor',
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
    'Plastering on outer sides',
    'Tiles work',
    'Painting and Finishing',
    'Plumbing & Sanitary,Electrification Works',
    'Doors & Windows Fixing Furniture work'
]

# Target variables
X_new = new_data[numerical_columns.columns.difference(stage_columns)].drop(columns=['floor', 'total_area_of_project'])
#y_total_days = new_data['total_days']
#y_time_taken_at_each_stage = new_data[stage_columns]  # Replace with your actual stage columns

for stage in stage_columns:
    stage_model = stage_models.get(stage)  # Use get to avoid KeyError
    if stage_model:
        new_data[stage] = stage_model.predict(new_data[X_new.columns])
    else:
        # If the stage column isn't in stage_models, fill it with NaN
        new_data[stage] = np.nan

for index, row in new_data.iterrows():
    cumulative_duration = 0
    for stage in stage_columns:
        stage_duration = row[stage]
        stage_start_date = row['actual_commencement_date'] + pd.Timedelta(days=cumulative_duration)
        stage_end_date = stage_start_date + pd.Timedelta(days=stage_duration)

        if stage_start_date <= current_date < stage_end_date:
            elapsed_duration = (current_date - stage_start_date).days
            remaining_duration = max(0, stage_duration - elapsed_duration)
            new_data.at[index, stage] = remaining_duration
            cumulative_duration += remaining_duration
            break
        else:
            cumulative_duration += stage_duration

new_data = adjust_stage_durations(new_data, stage_columns)

stage_materials = {
    'Cleaning & survey': [],
    'Excavation,leveling & P.C.C for Basement B1': ['cement', 'sand', 'aggregates'],
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring': ['TMT bar', 'cement', 'sand', 'aggregates'],
    'Slab of B (bottom)': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Ground Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '1st floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '1st Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at Basement to Ground Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor': ['circuit pipe', 'Cpvc', '&Pvc pipe', 'cement', 'sand'],
    '2nd floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '2nd Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at 1st Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 1st floor': ['circuit pipe', 'Cpvc', '&Pvc pipe', 'cement', 'sand'],
    '3rd floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '3rd Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at 2nd Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor': ['cement', 'sand', 'circuit pipe', 'Cpvc', '&Pvc pipe'],
    '4th floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '4th Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at 3rd to 5th Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor': ['cement', 'sand', 'circuit pipe', 'Cpvc', '&Pvc pipe'],
    '5th floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '5th Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work of 3rd to 5th Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor': ['cement', 'sand', 'circuit pipe', 'Cpvc', '&Pvc pipe'],
    'Plastering on outer sides': ['cement', 'sand'],
    'Tiles work': ['Tiles'],
    'Painting and Finishing': ['paint', 'putty', 'primer'],
    'Plumbing & Sanitary,Electrification Works': ['Wire', 'switch', 'nozzle Bib cock', 'shower', 'kitchen accessories', 'lights', 'fans', 'sheets', 'basin', 'sink', 'etc.'],
    'Doors & Windows Fixing Furniture work': ['Furniture', 'hardware', 'glass']
}


# Initialize empty lists to store relevant_stage and relevant_columns
relevant_stage_list = []
relevant_columns_list = []

# Iterate through each row (project) in new_data
for index, row in new_data.iterrows():
    days_elapsed = (current_date - row['actual_commencement_date']).days
    total_days = row['total_days']

    relevant_stage = None
    relevant_columns = []

    for stage in reversed(stage_columns):  # Reverse the order of stage_columns

        if total_days is None or np.isnan(total_days):
            # Set relevant_stage and relevant_columns to None if total_days is missing
            relevant_stage = None
            relevant_columns = []
            break

        stage_duration = row[stage]
        total_days -= stage_duration

        if total_days <= days_elapsed:
            relevant_stage = stage
            relevant_columns = [stage for stage in stage_columns[stage_columns.index(stage):]]
            break

    relevant_stage_list.append(relevant_stage)
    relevant_columns_list.append(relevant_columns)

# Add the 'relevant_stage' and 'relevant_columns' columns to the new_data DataFrame
new_data['relevant_stage'] = relevant_stage_list
new_data['relevant_columns'] = relevant_columns_list

# Initialize empty DataFrame to store adjusted days data
adjusted_days_data = pd.DataFrame(columns=stage_columns)

# Iterate through each row (project) in new_data
for index, row in new_data.iterrows():
    current_stage = row['current_stage']
    relevant_columns = row['relevant_columns']

    if current_stage == 'completed':
        # For completed projects, set 'days' to NaN for all stage columns
        days_values = [np.nan] * len(stage_columns)
    elif current_stage == 'running':
        # For running projects, update 'days' based on relevant_columns
        days_values = [row[stage] if stage in relevant_columns else np.nan for stage in stage_columns]
    elif current_stage == 'upcoming':
        # For upcoming projects, keep the predicted 'days' values
        days_values = [row[stage] for stage in stage_columns]

    # Append the days_values to the adjusted_days_data
    adjusted_days_data = adjusted_days_data.append(pd.Series(days_values, index=stage_columns), ignore_index=True)


# Replace the original stage values in new_data with adjusted values
new_data[stage_columns] = adjusted_days_data

# Create a list to store the data
data = []

# Iterate through each row (project) in the new_data DataFrame
for index, row in new_data.iterrows():
    current_materials = []  # List to store materials for the current stage
    current_stage_start_date = current_date  # Initialize with the current date
    project_id = f'Project_{index + 1}'  # Generate a project_id

    for stage in stage_columns:
        stage_duration = row[stage]

        if not pd.isna(stage_duration):
            # Calculate the stage end date
            stage_end_date = current_stage_start_date + pd.Timedelta(days=stage_duration)

            # Format the dates for display
            start_date_str = current_stage_start_date.strftime('%m/%d/%Y')
            end_date_str = stage_end_date.strftime('%m/%d/%Y')

            # Check if there are materials for this stage
            materials = stage_materials.get(stage, [])

            if materials:
                # Store the data in the desired format including both start and end dates
                data.append([start_date_str, end_date_str, stage, ", ".join(materials), project_id, row['organisation_name'], row['project_res_no']])

            # Update the start date for the next stage
            current_stage_start_date = stage_end_date

# Create a DataFrame from the data
output_data = pd.DataFrame(data, columns=['Start Date', 'End Date', 'Stage', 'Recommended Materials', 'Project_ID', 'organisation_name', 'project_res_no'])

# Save the DataFrame to a CSV file
output_data.to_csv('kaboom12.csv', index=False)

Project 1: Adjusted Durations - [4.733333333333337, 18.929442282749687, 25.558949416342454, 14.201167315175072, 26.508845654993525, 10.414189364461741, 18.929442282749687, 10.414189364461741, 19.88081712062253, 17.038132295719866, 11.361089494163425, 20.82952010376135, 8.521284046692617, 25.558949416342454, 11.361089494163425, 19.88081712062253, 8.521284046692617, 18.929442282749687, 12.305654993514901, 20.82952010376135, 4.508845654993525, 66.26757457846949, 71.00486381322958, 56.79922178988318, 75.74370946822313, 42.60291828793778, 66.26757457846949]
Project 2: Adjusted Durations - [4.733333333333337, 18.929442282749687, 25.560700389105094, 14.201167315175072, 26.508845654993525, 10.41490272373541, 18.929442282749687, 10.41490272373541, 19.88081712062253, 17.03929961089496, 11.361089494163425, 20.82952010376135, 8.521284046692617, 25.560700389105094, 11.361089494163425, 19.88081712062253, 8.521284046692617, 18.929442282749687, 12.305654993514901, 20.82952010376135, 4.508845654993525,

<ipython-input-111-283ee05765ff>:235: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  adjusted_days_data = adjusted_days_data.append(pd.Series(days_values, index=stage_columns), ignore_index=True)
<ipython-input-111-283ee05765ff>:235: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  adjusted_days_data = adjusted_days_data.append(pd.Series(days_values, index=stage_columns), ignore_index=True)
<ipython-input-111-283ee05765ff>:235: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  adjusted_days_data = adjusted_days_data.append(pd.Series(days_values, index=stage_columns), ignore_index=True)
<ipython-input-111-283ee05765ff>:235: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 